In [1]:
from datetime import datetime
import os

import dask.dataframe as dd
from dask.distributed import Client
import json
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
from pyprojroot import here

In [2]:
def sent_list_to_list_set_tokens(list_of_sentences):
    return [set(word_tokenize(sent)) for sent in list_of_sentences]

In [3]:
from dask.distributed import Client

client = Client(n_workers=6, threads_per_worker=1, memory_limit='1GB')
client

Client Scheduler: tcp://127.0.0.1:64633 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 6 Memory: 6.00 GB


In [4]:
files = str(here("./data/db/working/kaggle/id_model_inputs/01-02-split_sentences.json.gzip"))
pth = os.path.join(files, "*.part")

In [5]:
paper_df = dd.read_json(pth, compression="gzip")

In [6]:
paper_df

,pid,num_authors,title,text,text_sent_lower
npartitions=985,,,,,
,object,int64,object,object,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [7]:
paper_df.head()

,pid,num_authors,title,text,text_sent_lower
0,PMC1054884,7,Recombination Every Day: Abundant Recombinatio...,As increasing numbers of full-length viral seq...,[as increasing numbers of full-length viral se...
1,PMC1065028,1,Why can't I visit? The ethics of visitation re...,The sudden emergence of severe acute respirato...,[the sudden emergence of severe acute respirat...
2,PMC1065064,8,Prospective evaluation of an internet-linked h...,The rate of expansion of medical knowledge is ...,[the rate of expansion of medical knowledge is...
3,PMC1065120,4,Scanning the horizon: emerging hospital-wide t...,This series of articles provides regular surve...,[this series of articles provides regular surv...
4,PMC1065257,3,Characterization of the frameshift signal of E...,Programmed −1 ribosomal frameshifting (hereaft...,[programmed −1 ribosomal frameshifting (hereaf...


In [8]:
paper_df.tail()

,pid,num_authors,title,text,text_sent_lower
12,PMC7299382,60,Wastewater-Based Epidemiology: Global Collabor...,Severe acute respiratory syndrome coronavirus ...,[severe acute respiratory syndrome coronavirus...
13,PMC7299394,12,Proinflammatory Cytokines in the Olfactory Muc...,This was a case series of COVID-19 patients ad...,[this was a case series of covid-19 patients a...
14,PMC7299395,15,Novel One-Step Single-Tube Nested Quantitative...,The specificity evaluation panel preserved in ...,[the specificity evaluation panel preserved in...
15,PMC7299396,6,Biosensors for Managing the COVID-19 Cytokine ...,SARS-CoV-2 infects epithelial lung cells via s...,[sars-cov-2 infects epithelial lung cells via ...
16,PMC7299399,17,Toward Nanotechnology-Enabled Approaches again...,"Through millions of years of evolution, viruse...","[through millions of years of evolution, virus..."


In [9]:
paper_df['sent_set'] = paper_df['text_sent_lower'].map_partitions(lambda s: s.apply(sent_list_to_list_set_tokens), meta="object")

In [10]:
# paper_df.head()

In [11]:
start_time = datetime.now()

paper_df.to_json(str(here("./data/db/working/kaggle/id_model_inputs/01-03-tokenize_sentences.json.gzip", warn=False)), compression="gzip") # about 17 minutes

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:17:09.619429


In [12]:
client.shutdown()